In [1]:
import requests
from PIL import Image
import json
import sqlite3
import pandas as pd
from io import BytesIO



In [2]:
df = pd.read_json("./breed_list.json")

In [3]:

df['imperial_weight'] = df['weight'].apply(lambda x: x['imperial'])
df['metric_weight'] = df['weight'].apply(lambda x: x['metric'])
df['imperial_height'] = df['height'].apply(lambda x: x['imperial'])
df['metric_height'] = df['height'].apply(lambda x: x['metric'])

In [4]:
df['image_id'] = df['image'].apply(lambda x: x['id'])
df['image_url'] = df['image'].apply(lambda x: x['url'])
df['image_width'] = df['image'].apply(lambda x: x['width'])
df['image_height'] = df['image'].apply(lambda x: x['height'])

In [5]:
df.drop(columns=['weight', 'height', 'origin', 'country_code', 'description', 'history', 'image' ], inplace=True)

In [ ]:

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

session = requests.Session()
retries = Retry(total=3, backoff_factor=1, status_forcelist=[502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

def image_converter(url):
    try:
        res = session.get(url, timeout=5)
        if res.status_code == 200:
            print(f"Success: {url}")
            return res.content
    except Exception as e:
        print(f"Failed: {url} - {e}")
    return None

df['image_blob'] = df['image_url'].apply(image_converter)


Success: https://cdn2.thedogapi.com/images/BJa4kxc4X.jpg
Success: https://cdn2.thedogapi.com/images/hMyT4CDXR.jpg
Success: https://cdn2.thedogapi.com/images/rkiByec47.jpg
Success: https://cdn2.thedogapi.com/images/1-7cgoZSh.jpg
Success: https://cdn2.thedogapi.com/images/26pHT3Qk7.jpg
Success: https://cdn2.thedogapi.com/images/BFRYBufpm.jpg
Success: https://cdn2.thedogapi.com/images/33mJ-V3RX.jpg
Success: https://cdn2.thedogapi.com/images/-HgpNnGXl.jpg
Success: https://cdn2.thedogapi.com/images/dW5UucTIW.jpg
Success: https://cdn2.thedogapi.com/images/pk1AAdloG.jpg
Success: https://cdn2.thedogapi.com/images/sqQJDtbpY.jpg
Success: https://cdn2.thedogapi.com/images/Bymjyec4m.jpg
Success: https://cdn2.thedogapi.com/images/_gn8GLrE6.jpg
Success: https://cdn2.thedogapi.com/images/S14n1x9NQ.jpg
Success: https://cdn2.thedogapi.com/images/HkC31gcNm.png
Success: https://cdn2.thedogapi.com/images/rJIakgc4m.jpg
Success: https://cdn2.thedogapi.com/images/SkmRJl9VQ.jpg
Success: https://cdn2.thedogapi

In [ ]:
# def image_converter(x):
#     print(1)
#     res = requests.get(x)
#     return res.content

# df['image_blob'] = df['image_url'].apply(image_converter)

In [7]:
df.to_sql('breeds', con=sqlite3.connect('dog_breeds_with_image.db'), if_exists='replace', index=False)


172

In [ ]:
con = sqlite3.connect('dog_breeds_with_image.db')
cur = con.cursor()





In [11]:
cur.execute('select image_blob from breeds')
rows = cur.fetchone()

In [12]:
img = Image.open(BytesIO(rows[0]))
img.show()